In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import glob
import pandas as pd
import netCDF4
import scipy
import numpy as np

In [2]:
directory_noDA = '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/'
directory_DA = '/data/c-quilo/CMCC/DA/SHYFEM_ASSIM_BIVARIATE/'
list_of_paths_noDA = sorted(glob.glob(directory_noDA + '/*'))
list_of_paths_DA = sorted(glob.glob(directory_DA + '/*'))
print(list_of_paths_noDA)

['/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0000.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0001.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0002.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0003.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0004.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0005.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0006.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0007.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0008.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0009.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0010.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0011.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0012.nos.nc', '/data/c-quilo/CMCC/NoDA/SHYFEM_NO_ASSIM/sst_saniv1_chunk_0013.

In [3]:
sst_noDA = np.zeros((365*24, 90351))
count = 0
for file in list_of_paths_noDA:
    data_noDA = xr.open_dataset(file)
    sst_noDA[24*count:24*(count+1), :] = data_noDA.temperature
    count += 1

sst_DA = np.zeros((365*24, 90351))
count = 0
for file in list_of_paths_DA:
    data_DA = xr.open_dataset(file)
    sst_DA[24*count:24*(count+1), :] = data_DA.temperature
    count += 1

In [6]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

trainingData = sst_noDA

Variational autoencoder
Need to create 2 VAE, one to reduce the noDA staset, and another for the DA dataset

In [20]:
class Sampling(keras.layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [21]:
nFeatures = trainingData.shape[1] #+1 to account for droplet diameter size
latent_dim = nFeatures
initNNodes = 512

alpha = 0.3
encoder_inputs = keras.Input(shape=nFeatures)
nNodes = initNNodes

enc = keras.layers.Dense(initNNodes)(encoder_inputs)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)
enc = keras.layers.Dense(initNNodes)(enc)
enc = keras.layers.LeakyReLU(alpha)(enc)
enc = keras.layers.Dropout(0.5)(enc)
enc = keras.layers.BatchNormalization()(enc)

z_mean = keras.layers.Dense(latent_dim, name="z_mean")(enc)
z_log_var = keras.layers.Dense(latent_dim, name="z_log_var")(enc)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 90351)]      0                                            
______________________________________________________________________________________

In [22]:
latent_inputs = keras.Input(shape=(latent_dim,))
n = latent_dim

dec = keras.layers.Dense(initNNodes)(latent_inputs)
dec = keras.layers.LeakyReLU(alpha)(dec)
dec = keras.layers.Dropout(0.5)(dec)
dec = keras.layers.BatchNormalization()(dec)
dec = keras.layers.Dense(initNNodes)(dec)
dec = keras.layers.LeakyReLU(alpha)(dec)
dec = keras.layers.Dropout(0.5)(dec)
dec = keras.layers.BatchNormalization()(dec)

decoder_outputs = keras.layers.Dense(nFeatures, activation='sigmoid')(dec)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 90351)]           0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               46260224  
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 512)               2048      
_________________________________________________________________
dense_19 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 512)               0   

In [23]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #Mean squared Error as loss
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction)
                )
            )

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [26]:
tf.random.set_seed(42)

def scaler(x, xmin, xmax, min, max):
    scale = (max - min) / (xmax - xmin)
    xScaled = scale * x + min - xmin * scale
    return xScaled
min_ls = np.min(trainingData, 0)
print(min_ls)
max_ls = np.max(trainingData, 0)
print(max_ls)
min = 0
max = 1
meanData = np.mean(trainingData, 0)
stdData = np.std(trainingData, 0)

data = scaler(trainingData, min_ls, max_ls, min, max)

print(data.shape)
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Nadam())
vae.fit(data, epochs=2000, batch_size=512)

[10.03633384  9.88662116  9.76722944 ... 14.16660142 14.23843201
 14.19209752]
[31.05617823 31.02761777 30.9880134  ... 31.09058508 31.17212671
 31.13100867]
(8760, 90351)


2022-10-17 22:26:30.161786: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-10-17 22:26:30.188089: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3504000000 Hz


Epoch 1/2000
16/18 [=========================>....] - ETA: 6s - loss: 2124219156079.9851 - reconstruction_loss: 139.0680 - kl_loss: 1387293966336.0000